# TRABAJO PRÁCTICO 2 - MACHINE LEARNING
### INTEGRANTES: MORENO, AGOSTINA; RODRÍGUEZ, NICOLÁS; SORATI, GASTÓN

#### Continuando con el trabajo realizado en el Trabajo Práctico 1, utilizaremos el dataset de Destinos AirBNB (kaggle.com) con la finalidad de poder predecir al momento que un usuario o usuaria se registra, si efectivamente terminará haciendo una reserva en el sitio (lo que le puede servir al negocio para mostrarle anuncios, ofertas o promociones específicas, por ejemplo).

#### Para ello, desarrollaremos y probaremos modelos con distintos algoritmos de machine learning a fin de obtener el que mejor cumpla con el objetivo de negocio.

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import itertools
import warnings
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, matthews_corrcoef
from sklearn.model_selection import train_test_split
from keras import backend as K
from matplotlib import pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Activation, Input, Dropout

from sklearn.preprocessing import minmax_scale
warnings.filterwarnings("ignore")

In [0]:
data = pd.read_csv('train_users_2.csv')

In [0]:
data

#### En primer término aplicaremos a la base de datos el preprocesamiento desarrollado y justificado en el Trabajo Práctico 1:

In [0]:
data['reserva'] = data.country_destination != 'NDF'
data['reserva'].value_counts()

In [0]:
# Cuando se hacia el one-hot encoder de esta variable daba un error porque tiene valores nulos por eso los elimino en la linea de abajo
data.first_affiliate_tracked.isnull().sum()

In [0]:
data = data.drop(data[data.first_affiliate_tracked.isnull()].index)

#### Con respecto a la variable 'edad', realizamos el mismo binning que en el Trabajo Práctico 1, lo cual ahora nos será de utilidad para que el modelo no sobreentrene:

In [0]:
# Se eliminan los registros que tengan como edad menores de 18 y mayores de 100
data = data.drop(data[(data.age > 100)].index)
data = data.drop(data[(data.age < 18)].index)

In [0]:
# Se crea una nueva variable con el rango etario al cual pertenece segun la edad
limite = [18, 35, 65, np.inf]
categoria = ['Adulto Joven', 'Adulto', 'Adulto Mayor']

data['rango_etario'] = pd.cut(data['age'], bins=limite, labels=categoria, right=False)

In [0]:
# Se verifica cuantos no ingresan dentro de un rango etario y quedaron en null
data.rango_etario.isnull().sum()

In [0]:
# Se crea una nueva categoria y se incluye en esta todos los que tengan valores null
data['rango_etario'] = data['rango_etario'].cat.add_categories('Edad Desconocida').fillna('Edad Desconocida')

In [0]:
# Se elimina la columna edad
data.drop("age",axis = 1,inplace = True)

In [0]:
#Conversion de variables
data_tree = data.copy()

data['sex_m'] = data.gender == 'MALE'
data['sex_f'] = data.gender == 'FEMALE'
data['sex_d'] = data.gender == '-unknown-'
data['sex_o'] = data.gender == 'OTHER'
data.drop("gender",axis = 1,inplace = True)

data['signup_aw'] = data.signup_app == 'Web'
data['signup_ai'] = data.signup_app == 'iOS'
data['signup_am'] = data.signup_app == 'Moweb'
data['signup_aa'] = data.signup_app == 'Android'
data.drop("signup_app",axis = 1,inplace = True)

data['signup_mb'] = data.signup_method == 'basic'
data['signup_mf'] = data.signup_method == 'facebook'
data['signup_mg'] = data.signup_method == 'google'
data.drop("signup_method",axis = 1,inplace = True)

In [0]:
data

#### Aplicamos One Hot Encoder a las variables, al igual que en el Trabajo Práctico 1:

In [0]:
# one-hot encoder para language
for language in data.language.unique():
    data[language.lower().replace(" ", "_")] = data.language == language 
data.drop("language",axis = 1,inplace = True)

# one-hot encoder para first_affiliate_tracked
for first_affiliate_tracked in data.first_affiliate_tracked.unique():
    data[first_affiliate_tracked.replace(" ", "_")] = data.first_affiliate_tracked == first_affiliate_tracked 
data.drop("first_affiliate_tracked",axis = 1,inplace = True)
#data.gender = data.gender.astype(float)

# one-hot encoder para affiliate_provider
for affiliate_provider in data.affiliate_provider.unique():
    data[affiliate_provider.lower().replace(" ", "_")] = data.affiliate_provider == affiliate_provider 
data.drop("affiliate_provider",axis = 1,inplace = True)

# one-hot encoder para first_device_type
for first_device_type in data.first_device_type.unique():
    data[first_device_type.lower().replace(" ", "_")] = data.first_device_type == first_device_type 
data.drop("first_device_type",axis = 1,inplace = True)

# one-hot encoder para first_browser
for first_browser in data.first_browser.unique():
    data[first_browser.lower().replace(" ", "_")] = data.first_browser == first_browser 
data.drop("first_browser",axis = 1,inplace = True)

# one-hot encoder para first_browser
for rango_etario in data.rango_etario.unique():
    data[rango_etario.lower().replace(" ", "_")] = data.rango_etario == rango_etario 
data.drop("rango_etario",axis = 1,inplace = True)

In [0]:
# Se eliminan columnas que no se van a utilizar
data.drop("id",axis = 1,inplace = True)
data.drop("date_account_created",axis = 1,inplace = True)
data.drop("timestamp_first_active",axis = 1,inplace = True)
data.drop("date_first_booking",axis = 1,inplace = True)
data.drop("signup_flow",axis = 1,inplace = True)
data.drop("affiliate_channel",axis = 1,inplace = True)
data.drop("country_destination",axis = 1,inplace = True)

##data = data.drop(data[(data.age > 100)].index)
#data = data.drop(data[(data.age < 18)].index)

#data['age'].apply(pd.to_numeric, errors='coerce')

In [0]:
#Separamos el conjunto de datos en: train, test y validation
train, not_train = train_test_split(data, test_size=0.4)
validation, test = train_test_split(not_train, test_size=0.5)

In [0]:
#Definimos las columas a utilizar en las predicciones (eliminamos la columna target)
INPUTS_COLUMNS = data.columns.values
index = np.where(INPUTS_COLUMNS == "reserva")[0]
INPUTS_COLUMNS = np.delete(INPUTS_COLUMNS,index[0])
INPUTS_COLUMNS

## MÉTRICA DE EVALUACIÓN: F1 SCORE

#### Para medir nuestros modelos utilizaremos la métrica F1 Score porque nos permite establecer una relación entre lás métricas Precission y Recall. Precission es útil en los casos en los que queremos estar seguros de hacer una predicción correcta. Recall es útil si queremos asegurararnos de que la prediccion sea correcta en la mayoria de los casos.

#### Cabe destacar que estas dos métricas funcionan bien cuando hay menor cantidad de casos positivos en la variable target (como sucede en nuestro dataset con la cantidad de reserva de usuarios).

#### Además, contrastaremos los resultados obtenidos con la métrica Matthews Coefficient Correlation (MCC) ya que según cierta bibliografía es más informativa que F1 Score en problemas de clasificación binaria, como es nuestro caso (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5721660/#Sec9)

#### Para realizar la evaluación en sí, utilizaremos la siguiente función:

In [0]:
def evaluate_model(model, extract_inputs_function, extract_outputs_function, tree ,include_validation=False, nn=False):
    if tree:
        sets = [('train', train_tree), ('test', test_tree), ('validation', validation_tree)] #Tree
    else:
        sets = [('train', train), ('test', test), ('validation', validation)] 
    if include_validation:
        sets.append(('validation', validation))
        
    for set_name, set_data in sets:
        inputs = extract_inputs_function(set_data)
        outputs = extract_outputs_function(set_data)
        predictions = model.predict(inputs)
        if nn:
            predictions = np.around(predictions)
        
        print(set_name, '#' * 80)
        
        # print metrics
        
        print('Precision:', precision_score(outputs, predictions))
        print('Recall:', recall_score(outputs, predictions))
        print('F1Score:', f1_score(outputs, predictions))
        print('MCC:', matthews_corrcoef(outputs, predictions))
        
        print()
        
        # plot confussion matrix
        
        plt.figure(figsize=(3,4))
        
        plt.xticks([0, 1], ['reserva', 'no reserva'], rotation=45)
        plt.yticks([0, 1], ['no reserva', 'reserva'])
        plt.xlabel('Predicted class')
        plt.ylabel('True class')

        plt.title(set_name)

        plt.imshow(
            confusion_matrix(outputs, predictions), 
            cmap=plt.cm.Blues, 
            interpolation='nearest',
        )

        plt.show()

## MODELO 1: LINEAR SVC

#### En primer lugar utilizaremos el algoritmo LinearSVC, ya que según la bibliografía de scikit-learn es el más apropiado para problemas de clasificación como el nuestro (https://scikit-learn.org/stable/_static/ml_map.png):

In [0]:
from sklearn import svm
from sklearn.svm import LinearSVC

def lsvc_extract_inputs(data):
    return data[INPUTS_COLUMNS].values.astype(np.float64)
   
def lsvc_extract_outputs(data):
    return data.reserva.values

In [0]:
lsvc_model = LinearSVC(C = 1.0)
lsvc_model.fit(lsvc_extract_inputs(train),lsvc_extract_outputs(train),)

In [0]:
evaluate_model(lsvc_model, lsvc_extract_inputs, lsvc_extract_outputs, False)

**Conclusión:**

En el 66,52% de los casos que el modelo LinearSVC predijo que un usuario reservaría sobre el conjunto 'test', ese usuario efectivamente lo hizo. En el restante 33,48% de los casos se trataba de usuarios que en realidad no reservaron.

Además, identificó correctamente al 53,65% de los usuarios que reservaron, mientras que en el 46,35% de los casos se equivocó al predecir que no reservarían.

Su F1 Score es de 0,594 y su MCC de 0.36.

Consideramos que el modelo funciona bien, ya que considerando que el dataset tiene más casos que no reservan en relación a los que sí lo hacen, puedo identificar al 53,65% de los usuarios que se registran y harán una reserva.

## MODELO 2: REGRESIÓN LOGÍSTICA

#### Como segunda opción utilizaremos uno de los algoritmos más sencillos, y es útil para problemas de clasificación binaria como el nuestro.

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [0]:
def lr_extract_inputs(data):
    return data[INPUTS_COLUMNS].values. astype(np.float64)
   
def lr_extract_outputs(data):
    return data.reserva.values

In [0]:
lr_model = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=30, solver='liblinear')),
])

In [0]:
lr_model.fit(
    lr_extract_inputs(train),
    lr_extract_outputs(train),
)

In [0]:
evaluate_model(lr_model, lr_extract_inputs, lr_extract_outputs, False)

**Conclusión:**
Podemos concluir con que este modelo no es el apropiado para nuestro caso ya que los valores que expresan las métricas son bajos. 
Observamos una precisión de 60% con este modelo, esto puede deberse a nuestro conjunto de datos no es linealmente divisible.

En el 66,37% de los casos que el modelo de Regresión Lineal predijo que un usuario reservaría sobre el conjunto 'test', ese usuario efectivamente lo hizo.

Además, identificó correctamente al 53,61% de los usuarios que reservaron.

Expresado como F1 Score su valor numérico es de 0,593 y el MCC de 0.358.

Sus métricas son inferiores a los valores de LinearSVC, por lo que este modelo no será priorizado.



## MODELO 3: GRADIENT BOOSTED TREES

In [0]:
def tree_extract_inputs(data):
    return data[INPUTS_COLUMNS].values.astype(np.float64)
   
def tree_extract_outputs(data):
    return data.reserva.values

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

#### Aplicamos una profunidad máxima de 8 niveles para evitar que el modelo se aprenda el recorrido exacto para caso de prueba:

In [0]:
boost_model = GradientBoostingClassifier(max_depth=8)

In [0]:
boost_model.fit(
    tree_extract_inputs(train),
    tree_extract_outputs(train),
)

In [0]:
evaluate_model(boost_model, tree_extract_inputs, tree_extract_outputs, False)

**Conclusión:**

En el 65,28% de los casos que el modelo de Gradient Boosted Trees predijo que un usuario reservaría sobre el conjunto 'test', ese usuario efectivamente lo hizo. 

Además, identificó correctamente al 56,9% de los usuarios que reservaron.

Su F1 Score es de 0,608 y su MCC de 0.364. Ambas métricas son superiores a las del modelo LinearSVC. Esto se debe a que su métrica de Recall mejora respecto al anterior modelo, mientras que su valor de Precission disminuye en menor medida.

Consideramos que el modelo actual funciona mejor que LinearSVC.

## MODELO 4: ADABOOSTING

In [0]:
from sklearn.ensemble import AdaBoostClassifier

def ada_extract_inputs(data):
    return data[INPUTS_COLUMNS].values.astype(np.float64)
   
def ada_extract_outputs(data):
    return data.reserva.values

In [0]:
ada = AdaBoostClassifier(n_estimators=100, random_state=0)

ada.fit(ada_extract_inputs(train),ada_extract_outputs(train),)

In [0]:
evaluate_model(ada, ada_extract_inputs, ada_extract_outputs, False)

**Conclusión:**

En el 66,21% de los casos que el modelo de AdaBoosting predijo que un usuario reservaría sobre el conjunto 'test', ese usuario efectivamente lo hizo.

Además, identificó correctamente al 53,57% de los usuarios que reservaron.


Su F1 Score es de 0,592 y su MCC de 0.357.

Este modelo presenta resultados similares al modelo LinearSVC, es decir, es similarmente inferior ante la alternativa de Gradient Boosted Trees.

## CONCLUSIONES FINALES:

El modelo de machine learning que desarrollamos en base al algoritmo de Gradient Boosted Trees nos permite identificar correctamente al 56,9% de los usuarios que harán una reserva en el sitio.

Habrá un 34,72% de usuarios que considero que harán una reserva, pero en realidad no la harán. Esto no tiene efectos negativos sobre el negocio, ya que no hay una pérdida por ofrecerle un anuncio o promoción que no usará (excepto el costo de oportunidad de haberle ofrecido otro anuncio diferente).

Su valor de F1 Score (0.608) refleja su capacidad de identificar correctamente a los usuarios que reservarán, sin preocuparnos demasiado por los verdaderos negativos (usuarios identificados correctamente como que no harán una reserva).

Su valor de MCC (0.364) es bueno, considerando el 0 una random guess y el 1 una predicción perfecta.